In [1]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dropout
import keras
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
import math

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC 
from augmentdata import data_augment
from sklearn.model_selection import GridSearchCV 

import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
def check_cm_others(y_actual,y_predict):
    from sklearn.metrics import confusion_matrix

    cm1 = confusion_matrix(y_actual,y_predict)
    print('Confusion Matrix : \n', cm1)

    total1=sum(sum(cm1))
    #####from confusion matrix calculate accuracy
    accuracy1=(cm1[0,0]+cm1[1,1])/total1
    print ('Accuracy : ', accuracy1)

    tn=cm1[0,0]
    fp=cm1[0,1]
    fn=cm1[1,0]
    tp=cm1[1,1]

    sensitivity1 = tp/(tp+fn)
#     print('Sensitivity : ', sensitivity1 )

    specificity1 = tn/(tn+fp)
#     print('Specificity : ', specificity1)
    recall=sensitivity1
    precision=tp/(tp+fp)

#     print("Precision = ",precision)
#     print("Recall = ",recall)

    f1_score=2*(precision*recall)/(precision+recall)
    
    g_mean=math.sqrt(sensitivity1*specificity1)

#     print("F1 score = ",f1_score)

    return accuracy1,sensitivity1,specificity1,precision,recall,f1_score,g_mean



In [3]:
df=pd.read_csv("data/creditcard.csv")

In [4]:
print(df["Class"].value_counts())


0    284315
1       492
Name: Class, dtype: int64


In [5]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
scaler = StandardScaler()
df['NormalizedAmount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df = df.drop(['Amount'], axis = 1)

In [7]:
df1 = df.pop('Class') 
df['Class']=df1 



In [8]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,NormalizedAmount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [9]:
X=df.iloc[:,:-1]
y=df.Class

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.4, random_state=10)


### Model to check on non augmented data

In [ ]:
X_train_no_aug = X_train.drop(['Time'], axis = 1)
X_test_no_aug = X_test.drop(['Time'], axis = 1)


In [ ]:
X_train_no_aug.shape

In [ ]:
model = Sequential([
Dense(input_dim = X_train_no_aug.shape[1], units = 16, activation = 'relu'),
Dense(units = 24, activation = 'relu'),
Dropout(0.5),
Dense(units = 20, activation = 'relu'),
Dense(units = 24, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train_no_aug, y_train, batch_size = 15, epochs = 80)

In [ ]:
print(model.evaluate(X_test_no_aug,y_test))

In [ ]:
y_predict=model.predict_classes(X_test_no_aug)
acc,sens,spec,prec,recall,f1_score,g_mean=check_cm_others(y_test,y_predict)


#                 print("Sensitivity = ",sens," Specificity = ",spec," precision=",prec," recall = ",recall," f1_score = ",f1_score," g_mean = ",g_mean )
print(" precision=",prec," recall = ",recall," f1_score = ",f1_score," g_mean = ",g_mean)

In [ ]:
X_train.shape

In [ ]:
print(sum(y_train==0),sum(y_train==1))

### SMOTE

In [11]:
y_train.head()

254399    0
172532    0
39515     0
250002    0
134355    0
Name: Class, dtype: int64

In [12]:
X_train_smote = X_train.drop(['Time'], axis = 1)
X_test_smote = X_test.drop(['Time'], axis = 1)

In [13]:

k=5

# samp_strategy_range=[0.83]


print(X_train_smote.shape)

rows=X_train_smote.shape[0]

(170884, 29)


In [18]:
model = Sequential([
Dense(input_dim = X_train_smote.shape[1], units = 16, activation = 'relu'),
Dense(units = 24, activation = 'relu'),
Dropout(0.5),
Dense(units = 20, activation = 'relu'),
Dense(units = 24, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [27]:
batchsize=20000
start=0
total_augmented=0
counts=0
for i in range(0,rows+1,batchsize):
    start=i
    stop=min((start+batchsize),rows)
    print(start,stop)
    X_train_smote_batch=X_train_smote.iloc[start:stop,:]
    y_train_smote_batch=y_train.iloc[start:stop]
    print(X_train_smote_batch.shape,y_train_smote_batch.shape)
#     print(X_train_smote_batch.head())
    # augmentation
    minority_count=sum(y_train_smote_batch==1)
    majority_count=sum(y_train_smote_batch==0)  
    print(" majority = ",majority_count,"minority = ",minority_count)
    imb_ratio=minority_count/majority_count
    sampling_ratio=(3*minority_count)/majority_count
    print("IR = ",imb_ratio,sampling_ratio)

    
    if minority_count>5:        
        print("Augmenting")

        daug = data_augment.DataAugment()
        sm = SMOTE(random_state=2,k_neighbors=k,sampling_strategy=sampling_ratio)
        X_train_smote_applied, y_train_smote_applied = sm.fit_sample(X_train_smote_batch, y_train_smote_batch.ravel())
        print("DOne augmenting")
        total_augmented+=(X_train_smote_applied.shape[0]-X_train_smote_batch.shape[0])
        print("Count augmented = ",X_train_smote_applied.shape[0]-X_train_smote_batch.shape[0])

        model.fit(X_train_smote_applied, y_train_smote_applied, batch_size = 15, epochs = 5)
        counts+=1
        print("After KNNOR Majority = ",sum(y_train_smote_applied==0)," Minority = ",sum(y_train_smote_applied==1))
        
    print("*****")

0 20000
(20000, 29) (20000,)
 majority =  19963 minority =  37
IR =  0.0018534288433602164 0.005560286530080649
Augmenting
DOne augmenting
Count augmented =  74
Epoch 1/5
20074/20074 [==============================] - 2s 92us/step - loss: 0.0058 - accuracy: 0.9989
Epoch 2/5
20074/20074 [==============================] - 2s 93us/step - loss: 0.0027 - accuracy: 0.9993
Epoch 3/5
20074/20074 [==============================] - 2s 94us/step - loss: 0.0015 - accuracy: 0.9996
Epoch 4/5
20074/20074 [==============================] - 2s 84us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 5/5
20074/20074 [==============================] - 2s 79us/step - loss: 0.0011 - accuracy: 0.9998
After KNNOR Majority =  19963  Minority =  111
*****
20000 40000
(20000, 29) (20000,)
 majority =  19955 minority =  45
IR =  0.0022550739163117012 0.006765221748935104
Augmenting
DOne augmenting
Count augmented =  90
Epoch 1/5
20090/20090 [==============================] - 1s 72us/step - loss: 0.0072 - accuracy: 0.99

In [30]:
X_test_knnor = X_test.drop(['Time'], axis = 1)

In [31]:
y_predict=model.predict_classes(X_test_knnor)
acc,sens,spec,prec,recall,f1_score,g_mean=check_cm_others(y_test,y_predict)


#                 print("Sensitivity = ",sens," Specificity = ",spec," precision=",prec," recall = ",recall," f1_score = ",f1_score," g_mean = ",g_mean )
print(" precision=",prec," recall = ",recall," f1_score = ",f1_score," g_mean = ",g_mean)

Confusion Matrix : 
 [[113706     27]
 [    45    145]]
Accuracy :  0.9993679941715018
 precision= 0.8430232558139535  recall =  0.7631578947368421  f1_score =  0.8011049723756906  g_mean =  0.8734853876796861


In [ ]:
'''
batch of 20000

Confusion Matrix : 
 [[113706     27]
 [    45    145]]
Accuracy :  0.9993679941715018
 precision= 0.8430232558139535  recall =  0.7631578947368421  
 f1_score =  0.8011049723756906  g_mean =  0.8734853876796861
 
'''

In [ ]:
'''
batch of 10000
k=5
Confusion Matrix : 
 [[113708     25]
 [    42    148]]
Accuracy :  0.9994118834651475
 precision= 0.8554913294797688  recall =  0.7789473684210526  f1_score =  0.8154269972451791  g_mean =  0.8824829435223552
'''

In [ ]:
print("Number of items augmented = ",total_augmented)